In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import DistanceMetric
from scipy.spatial.distance import mahalanobis
import os
import sys
sys.path.append('../models')
import ResNet as resnet
import DenseNet as densenet
import EfficientNet as efficientnet
import MobileNetV2 as mobilenet
import ViT as vit

In [2]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "2, 3"

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
##########################################
######### Configure Metric Setting ####### 
##########################################
batch_size = 64
weight_path = './weights/cifar10_resnet18_v3.pt'
model = torch.load(weight_path)
criterion = nn.CrossEntropyLoss()
member_path = '/media/data1/hyunjun/cifar-10/train/'
nonmember_path = '/media/data1/hyunjun/cifar-10/test/'
data = 'Cifar-10'
##########################################
##########################################

In [4]:
mean, std = (0.485, 0.456, 0.406), (0.229, 0.224, 0.225) 
if data == 'STL-10':
    mean, std = (0.485, 0.456, 0.406), (0.229, 0.224, 0.225)
elif data == 'Cifar-10':
    mean, std = (0.491, 0.482, 0.447), (0.247, 0.243, 0.262)
elif data == 'CelebA':
    mean, std = (0.485, 0.456, 0.406), (0.229, 0.224, 0.225)

trans = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

model.to(device)
model.eval()

nonmember_set = torchvision.datasets.ImageFolder(root=nonmember_path, transform=trans)
nonmember_loader = DataLoader(nonmember_set, batch_size=batch_size, shuffle=True, drop_last=False,)
member_set = torchvision.datasets.ImageFolder(root=member_path, transform=trans)
member_loader = DataLoader(member_set, batch_size=batch_size, shuffle=True, drop_last=False,)

classes = member_set.classes

In [5]:
latent_vectors = [[] for _ in range(len(classes))]

model.eval()
with torch.no_grad():
    for data in member_loader:
        images, labels = data[0].to(device), data[1].to(device)
        features, _ = model(images)

        deep_features = features.cpu().numpy().tolist()
        actual = labels.cpu().numpy().tolist()
        
        for i in range(len(actual)):
            latent_vectors[actual[i]].append(list(deep_features[i]))

In [6]:
print("#####################################")
print("### Calculating Cosine Similarity ###")
print("#####################################")


for i in range(len(latent_vectors)):
    cssm = cosine_similarity(np.array(latent_vectors[i]))
    print(f"class: {classes[i]}, max: {np.max(cssm):.4f}, min: {np.min(cssm):.4f}, mean: {np.mean(cssm):.4f}, var: {np.var(cssm):.4f}")


#####################################
### Calculating Cosine Similarity ###
#####################################
class: airplane, max: 1.0000, min: 0.4670, mean: 0.9887, var: 0.0007
class: automobile, max: 1.0000, min: 0.4366, mean: 0.9954, var: 0.0004
class: bird, max: 1.0000, min: 0.4519, mean: 0.9872, var: 0.0009
class: cat, max: 1.0000, min: 0.3732, mean: 0.9801, var: 0.0012
class: deer, max: 1.0000, min: 0.4221, mean: 0.9870, var: 0.0007
class: dog, max: 1.0000, min: 0.5390, mean: 0.9843, var: 0.0010
class: frog, max: 1.0000, min: 0.6490, mean: 0.9954, var: 0.0002
class: horse, max: 1.0000, min: 0.5320, mean: 0.9929, var: 0.0006
class: ship, max: 1.0000, min: 0.5017, mean: 0.9949, var: 0.0004
class: truck, max: 1.0000, min: 0.3656, mean: 0.9914, var: 0.0009


In [7]:
print("#####################################")
print("## Calculating Euclidean Distance ###")
print("#####################################")
dist = DistanceMetric.get_metric('euclidean')
for i in range(len(latent_vectors)):
    euclidean = dist.pairwise(np.array(latent_vectors[i]))
    print(f"class: {classes[i]}, max: {np.max(euclidean):.4f}, min: {np.min(euclidean):.4f}, mean: {np.mean(euclidean):.4f}, var: {np.var(euclidean):.4f}")

#####################################
## Calculating Euclidean Distance ###
#####################################
class: airplane, max: 23.6579, min: 0.0000, mean: 4.6271, var: 10.3973
class: automobile, max: 23.7376, min: 0.0000, mean: 4.2923, var: 9.6302
class: bird, max: 25.2600, min: 0.0000, mean: 4.4680, var: 10.2548
class: cat, max: 25.9025, min: 0.0000, mean: 4.4393, var: 10.4150
class: deer, max: 26.7854, min: 0.0000, mean: 4.8046, var: 11.8209
class: dog, max: 26.3767, min: 0.0000, mean: 4.7520, var: 12.1981
class: frog, max: 27.1133, min: 0.0000, mean: 4.4309, var: 10.8498
class: horse, max: 33.1783, min: 0.0000, mean: 5.6592, var: 17.1234
class: ship, max: 32.8538, min: 0.0000, mean: 5.9138, var: 18.2216
class: truck, max: 27.6072, min: 0.0000, mean: 5.0058, var: 12.9705


In [8]:
print("#####################################")
print("## Calculating Manhattan Distance ###")
print("#####################################")
dist = DistanceMetric.get_metric('manhattan')
for i in range(len(latent_vectors)):
    manhattan = dist.pairwise(np.array(latent_vectors[i]))
    print(f"class: {classes[i]}, max: {np.max(manhattan):.4f}, min: {np.min(manhattan):.4f}, mean: {np.mean(manhattan):.4f}, var: {np.var(manhattan):.4f}")

#####################################
## Calculating Manhattan Distance ###
#####################################
class: airplane, max: 72.9251, min: 0.0000, mean: 13.9846, var: 100.9342
class: automobile, max: 73.0398, min: 0.0000, mean: 13.1647, var: 94.1278
class: bird, max: 77.1027, min: 0.0000, mean: 13.3937, var: 97.4210
class: cat, max: 77.6582, min: 0.0000, mean: 13.1349, var: 96.5256
class: deer, max: 80.1264, min: 0.0000, mean: 14.1433, var: 108.2766
class: dog, max: 77.1610, min: 0.0000, mean: 13.9112, var: 107.9897
class: frog, max: 83.2022, min: 0.0000, mean: 13.4631, var: 103.8483
class: horse, max: 99.4058, min: 0.0000, mean: 16.8310, var: 155.9876
class: ship, max: 101.5884, min: 0.0000, mean: 18.1891, var: 177.1307
class: truck, max: 84.7396, min: 0.0000, mean: 15.2906, var: 126.3945


In [9]:
print("#####################################")
print("## Calculating Chebyshev Distance ###")
print("#####################################")
dist = DistanceMetric.get_metric('chebyshev')
for i in range(len(latent_vectors)):
    chebyshev = dist.pairwise(np.array(latent_vectors[i]))
    print(f"class: {classes[i]}, max: {np.max(chebyshev):.4f}, min: {np.min(chebyshev):.4f}, mean: {np.mean(chebyshev):.4f}, var: {np.var(chebyshev):.4f}")

#####################################
## Calculating Chebyshev Distance ###
#####################################
class: airplane, max: 11.9362, min: 0.0000, mean: 2.1647, var: 2.0477
class: automobile, max: 9.4198, min: 0.0000, mean: 1.7335, var: 1.3308
class: bird, max: 11.8477, min: 0.0000, mean: 2.0796, var: 2.0570
class: cat, max: 11.4281, min: 0.0000, mean: 1.9570, var: 1.6333
class: deer, max: 12.9324, min: 0.0000, mean: 2.2954, var: 2.4953
class: dog, max: 11.9001, min: 0.0000, mean: 2.1349, var: 2.2251
class: frog, max: 11.4640, min: 0.0000, mean: 1.9072, var: 1.8344
class: horse, max: 15.5476, min: 0.0000, mean: 2.5770, var: 3.3664
class: ship, max: 13.6685, min: 0.0000, mean: 2.4649, var: 2.9165
class: truck, max: 11.7420, min: 0.0000, mean: 2.0929, var: 1.9668
